# Exercise 3 Question 1

## 1. Mnist

In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

## 2. Diabetes

In [3]:
from sklearn.datasets import fetch_openml

diabetes = fetch_openml('diabetes', version=1, as_frame=False)
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

## 3. Iris

In [4]:
from sklearn.datasets import fetch_openml

diabetes = fetch_openml('iris', version=1, as_frame=False)
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

## 4. spambase

In [5]:
from sklearn.datasets import fetch_openml

diabetes = fetch_openml('spambase', version=1, as_frame=False)
diabetes.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

# Exercise 3 Question 2

In [6]:
import numpy as np

X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)

In [7]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....n_neighbors=3, weights=uniform;, score=0.972 total time=  35.4s
[CV 2/5] END ....n_neighbors=3, weights=uniform;, score=0.971 total time=  35.3s
[CV 3/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  35.7s
[CV 4/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  35.0s
[CV 5/5] END ....n_neighbors=3, weights=uniform;, score=0.970 total time=  35.3s
[CV 1/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  35.3s
[CV 2/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  33.8s
[CV 3/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  37.0s
[CV 4/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  35.3s
[CV 5/5] END ...n_neighbors=3, weights=distance;, score=0.971 total time=  33.9s
[CV 1/5] END ....n_neighbors=4, weights=uniform;, score=0.969 total time=  35.5s
[CV 2/5] END ....n_neighbors=4, weights=uniform;,

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5],
                          'weights': ['uniform', 'distance']}],
             verbose=3)

In [9]:
grid_search.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [10]:
grid_search.best_score_

0.9716166666666666

In [11]:
from sklearn.metrics import accuracy_score


y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.9714

# Exercise 3 Question 3

In [12]:
from scipy.ndimage.interpolation import shift

<ipython-input-12-c50379b1544a>:1: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [13]:
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [14]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [15]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [16]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9763